In [1]:
import sqlite3
import pandas as pd 
from itertools import *
import numpy as np
import json

In [2]:
'''Our objective is to make metapaths from our graph with end nodes as CNG, Word or Lemma and internal nodes as Distinct CNGs
or CNG Groups'''

'Our objective is to make metapaths from our graph with end nodes as CNG, Word or Lemma and internal nodes as Distinct CNGs\nor CNG Groups'

In [3]:
conn = sqlite3.connect('Extracted.db')

In [4]:
CNG = pd.read_sql(sql = 'SELECT CNG FROM Extracted GROUP BY CNG', con = conn)
#extracting all the distinct CNG's from the scentences

In [5]:
#These Json Files contain the CNG Groupings 
dictnoun = json.load(open("noungroups.json"))
dictverb = json.load(open("verbgroups.json"))

In [6]:
print(dictnoun)
print(dictverb)

{'instr. adj.': [92, 97, 102], 'fem': [30, 35, 40, 50, 55, 60, 70, 75, 80, 90, 95, 100, 110, 115, 120, 130, 135, 140, 150, 155, 160, 170, 175, 180], 'acc. pl.': [79, 80, 81, 82], 'acc. neutr.': [71, 76, 81], 'voc': [49, 50, 51, 54, 55, 56, 59, 60, 61], 'loc': [169, 170, 171, 172, 174, 175, 176, 177, 179, 180, 181, 182], 'loc. du.': [174, 175, 176, 177], 'nom. masc.': [29, 34, 39], 'dat. du.': [114, 115, 116, 117], 'abl. pl.': [139, 140, 141, 142], 'du': [34, 35, 36, 37, 54, 55, 56, 74, 75, 76, 77, 94, 95, 96, 97, 114, 115, 116, 117, 134, 135, 136, 137, 154, 155, 156, 157, 174, 175, 176, 177], 'voc. neutr.': [51, 56, 61], 'voc. masc.': [49, 54, 59], 'instr. pl.': [99, 100, 101, 102], 'acc. du.': [74, 75, 76, 77], 'instr': [89, 90, 91, 92, 94, 95, 96, 97, 99, 100, 101, 102], 'abl': [129, 130, 131, 132, 134, 135, 136, 137, 139, 140, 141, 142], 'nom. du.': [34, 35, 36, 37], 'voc. pl.': [59, 60, 61], 'instr. neutr.': [91, 96, 101], 'adj': [32, 37, 42, 57, 62, 72, 77, 82, 92, 97, 102, 112, 1

In [7]:
#creating a list which contains all distinct CNG Groups
CNGG = list(set(list(dictnoun.keys())).union(set(list(dictverb.keys()))))

In [8]:
#CNG_list contains all the possible entries for internal nodes, i.e. distinct CNG's + distinct CNG_Groups
CNG_list = np.array(CNG['CNG']).tolist()
CNG_list = list(set(CNG_list).union(set(CNGG)) )
#done using union of 2 sets

In [9]:
print(CNG_list)

[1, 2, 3, '6_pl', '13_fp', '6_tp', 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 'nom. pl.', '9_fp', 'sg', '4_fp', 48, 49, 50, 51, '7_tp', '30_sp', 54, 55, 56, '8_sp', 58, 59, 60, 61, '13_sp', '10_sp', 'loc. pl.', '12_du', '8_fp', 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 108, 109, 110, 111, 112, 'gen', 114, 115, 116, 117, 118, 119, 120, 121, 122, '4_du', '27_fp', 'sg_tp', '2_sp', '14_tp', 128, 129, 130, 131, 132, '29_tp', 134, 135, 136, 137, 138, 139, 140, 141, 142, 'sp', 'pl', '27_pl', '3_sg', 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 'du', 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, '25_sg', 'voc. sg.', 'abl. sg.', '30_tp', '17_pl', '8_pl', '14_sp', '3_fp', '6_fp', '13_tp', 'voc. pl.', 'acc. masc.', '28_du', '4_pl', 'nom. adj.', '2_pl', '10_fp', '28_sg', 'instr. du.', '9_pl', '5_fp', 'voc. neutr.', '12_sp', '11_sg', '7_fp', '2

In [10]:
#Internal nodes will be of size 0, 1 or 2 
#0 for metapath of size 2 as it contains only External nodes, 1 for metapath of size 3 and 2 for metapath of size 4.
x = product(CNG_list,repeat = 2)
#product is a function of itertools which generates permutations with replacement.
#we need permuations as order will matter in the metapath as probabilty of going from one node to another depends on direction of metapath.
Internalnode2 = [i for i in x]
Internalnode1 = CNG_list

In [11]:
#External nodes will only be Lemma, Word or CNG as we are building the skeleton of the metapaths
x = product(["Lemma","Word","CNG"], repeat = 2)
Externalnodes = [i for i in x]

In [12]:
#creating metapaths as list of lists
metapath2 = Externalnodes 
metapath3 = [[]]
metapath4= [[]]
for i in Externalnodes:
    for j in Internalnode1:
        metapath3.append([i[0], j, i[1]])
        #adding metapaths of size 3 according to our defination
        
    for k in Internalnode2:
        metapath4.append([i[0], k[0], k[1], i[1]])
        #similarly adding metapaths of size 4

In [13]:
#to check
len(metapath4)

1962802

In [14]:
#storing the values in a dataframe so that I can dump them to sql db 
df4 = pd.DataFrame(data = metapath4, columns = ['node1', 'node2', 'node3', 'node4'])
print(df4.head())
df3 = pd.DataFrame(data = metapath3, columns = ['node1', 'node2', 'node3'])
print(df3.head())
df2 = pd.DataFrame(data = metapath2, columns = ['node1', 'node2'])
print(df2.head())

   node1 node2  node3  node4
0   None  None   None   None
1  Lemma     1      1  Lemma
2  Lemma     1      2  Lemma
3  Lemma     1      3  Lemma
4  Lemma     1  17_du  Lemma
   node1  node2  node3
0   None   None   None
1  Lemma      1  Lemma
2  Lemma      2  Lemma
3  Lemma      3  Lemma
4  Lemma  17_du  Lemma
   node1  node2
0  Lemma  Lemma
1  Lemma   Word
2  Lemma    CNG
3   Word   Word
4   Word    CNG


In [15]:
#dumping to sql
df4.to_sql('metapath4', conn)
df3.to_sql('metapath3', conn)
df2.to_sql('metapath2', conn)

In [17]:
conn.close()
